In [1]:
# expected returns are one of the main inputs for mean-variance optimisation, which is estimated with 
# historical data, which may be considered the main flaw in mean-variance optimisation. 

In [2]:
# PyPortfolioOpt expected_returns module provides functions for estimating the expected returns of assets:
# general return model function
# mean historical return
# exponentially weighted mean historical return
# CAPM estimate of returns 

In [11]:

import pandas as pd
import numpy as np
import cvxpy as cp
from pypfopt import risk_models
from pypfopt import expected_returns
from pypfopt import EfficientFrontier
from pypfopt import objective_functions
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices
from pypfopt import HRPOpt
from pypfopt import CLA
from pypfopt import black_litterman
from pypfopt import BlackLittermanModel
from pypfopt import plotting

In [12]:
df = pd.read_csv("br_stocks.csv", parse_dates=True, index_col="Date")

In [23]:
# the following command calculate the annualised mean (daily) historical return from input (daily) asset prices. 
returns = expected_returns.mean_historical_return(df)
returns.head(17)

ITSA4     0.180511
GOAU4     0.320892
CMIG3     0.176569
PETR4     0.234102
TOTS3    -0.029422
USIM5     0.322292
LAME4     0.124766
BBAS3     0.213770
CIEL3    -0.298643
LIGT3     0.092760
ABEV3    -0.043279
KLBN11    0.058826
VVAR3     0.443780
CVCB3     0.027951
GOLL4     0.414581
EMBR3    -0.230034
CSNA3     0.397387
dtype: float64

In [27]:
# the following command calculate the exponentially-weighted (daily) historical returns 
# giving higher weight to more recent data - exponential moving average is a simple improvement over the mean 
# historical return. It gives more credence to recent returns and 
# thus aims to increase the relevance of estimates. 
returns2 = expected_returns.ema_historical_return(df)
returns2.head(17)

ITSA4     0.033768
GOAU4     0.740648
CMIG3     0.182472
PETR4     0.118500
TOTS3     0.175180
USIM5     0.832620
LAME4     0.281752
BBAS3     0.078480
CIEL3    -0.323248
LIGT3     0.398447
ABEV3    -0.026861
KLBN11    0.471874
VVAR3     1.900324
CVCB3    -0.213888
GOLL4     0.529852
EMBR3    -0.322206
CSNA3     1.355181
Name: 2020-10-23 00:00:00, dtype: float64

In [28]:
# the following command compute a return estimate using the CAPM. Under the CAPM, asset returns are equal to 
# the market returns plus a eta eterm encoding the relative risk of the asset
returns3 = expected_returns.capm_return(df)
returns3.head(17)

ITSA4     0.169530
GOAU4     0.275060
CMIG3     0.236142
PETR4     0.258651
TOTS3     0.152013
USIM5     0.307661
LAME4     0.194876
BBAS3     0.249809
CIEL3     0.178437
LIGT3     0.227808
ABEV3     0.118046
KLBN11    0.092036
VVAR3     0.271635
CVCB3     0.252626
GOLL4     0.383218
EMBR3     0.158225
CSNA3     0.297387
Name: mkt, dtype: float64

In [30]:
# the following command calculate returns given prices
returns4 = expected_returns.returns_from_prices(df)
returns4.head(17)

,ITSA4,GOAU4,CMIG3,PETR4,TOTS3,USIM5,LAME4,BBAS3,CIEL3,LIGT3,ABEV3,KLBN11,VVAR3,CVCB3,GOLL4,EMBR3,CSNA3
Date,,,,,,,,,,,,,,,,,
2015-10-27,-0.008152,-0.043165,-0.023480,-0.031847,0.002289,0.000000,-0.002776,0.013464,0.009184,-0.011858,0.006606,-0.009195,0.000000,-0.016772,0.002817,0.047780,-0.008064
2015-10-28,-0.006849,-0.026316,0.016973,0.046053,-0.029141,-0.042763,-0.001670,-0.016908,-0.014407,-0.008800,-0.008582,-0.010454,-0.153153,0.036247,0.000000,-0.001077,-0.016260
2015-10-29,-0.015172,-0.030888,-0.022253,-0.042767,-0.020306,-0.034365,-0.059150,-0.047912,-0.043344,-0.003228,-0.032587,-0.017504,-0.117021,-0.005487,-0.019663,0.007908,-0.109504
2015-10-30,0.008403,0.035857,0.018492,0.013141,0.026434,-0.010676,-0.008899,0.030323,-0.020110,0.038866,0.004737,0.001827,0.164659,0.006897,0.017192,0.008559,0.006960
2015-11-03,0.043055,0.080769,0.055866,0.099870,0.067309,0.053957,0.073010,0.088291,0.033654,0.009353,0.026716,0.057912,0.103448,0.027397,0.030986,0.040311,0.165899
2015-11-04,0.006658,-0.035587,0.030424,-0.047170,-0.028516,-0.006826,-0.011155,0.004028,-0.015353,0.027027,0.000000,-0.008621,-0.015625,0.006667,0.010929,-0.038069,-0.019763
2015-11-05,0.015873,0.000000,0.023106,0.003713,0.037257,-0.003437,0.021997,0.048710,0.020164,-0.030075,-0.007143,-0.017391,-0.123810,0.006623,0.027027,0.009541,-0.002016
2015-11-06,-0.023437,-0.036900,-0.066499,-0.035758,-0.005169,-0.044828,-0.043599,-0.049180,-0.012912,-0.034884,-0.025180,0.001770,0.000000,-0.032895,-0.047368,-0.016101,0.010101
2015-11-09,-0.020000,0.007663,-0.005376,0.011509,0.009298,-0.018050,-0.001153,-0.013793,-0.022423,0.020884,-0.014233,-0.015901,-0.003623,0.000000,-0.019337,-0.012807,0.012000


In [33]:
# the following command calculate the log returns given prices
returns5 = expected_returns.log_returns_from_prices(df)
returns5.head(17)

,ITSA4,GOAU4,CMIG3,PETR4,TOTS3,USIM5,LAME4,BBAS3,CIEL3,LIGT3,ABEV3,KLBN11,VVAR3,CVCB3,GOLL4,EMBR3,CSNA3
Date,,,,,,,,,,,,,,,,,
2015-10-27,-0.008185,-0.044125,-0.023760,-0.032365,0.002287,0.000000,-0.002780,0.013374,0.009142,-0.011928,0.006584,-0.009237,0.000000,-0.016914,0.002813,0.046674,-0.008097
2015-10-28,-0.006873,-0.026668,0.016831,0.045024,-0.029574,-0.043704,-0.001672,-0.017053,-0.014512,-0.008839,-0.008619,-0.010509,-0.166236,0.035606,0.000000,-0.001078,-0.016394
2015-10-29,-0.015289,-0.031375,-0.022504,-0.043709,-0.020515,-0.034969,-0.060972,-0.049097,-0.044311,-0.003234,-0.033129,-0.017659,-0.124454,-0.005502,-0.019859,0.007877,-0.115977
2015-10-30,0.008368,0.035229,0.018323,0.013055,0.026090,-0.010733,-0.008938,0.029872,-0.020315,0.038130,0.004726,0.001826,0.152428,0.006873,0.017046,0.008523,0.006936
2015-11-03,0.042154,0.077673,0.054361,0.095192,0.065141,0.052551,0.070468,0.084608,0.033100,0.009310,0.026365,0.056297,0.098440,0.027029,0.030516,0.039520,0.153492
2015-11-04,0.006636,-0.036236,0.029970,-0.048319,-0.028930,-0.006849,-0.011217,0.004020,-0.015472,0.026668,0.000000,-0.008658,-0.015749,0.006645,0.010870,-0.038813,-0.019961
2015-11-05,0.015748,0.000000,0.022843,0.003706,0.036579,-0.003443,0.021759,0.047561,0.019963,-0.030537,-0.007169,-0.017544,-0.132172,0.006601,0.026668,0.009495,-0.002018
2015-11-06,-0.023716,-0.037598,-0.068813,-0.036413,-0.005182,-0.045864,-0.044578,-0.050431,-0.012996,-0.035507,-0.025502,0.001768,0.000000,-0.033448,-0.048527,-0.016232,0.010050
2015-11-09,-0.020202,0.007634,-0.005390,0.011443,0.009255,-0.018215,-0.001154,-0.013889,-0.022678,0.020668,-0.014335,-0.016029,-0.003630,0.000000,-0.019526,-0.012890,0.011929


In [32]:
# the following command calculate pseudo-prices given returns. According to PyPortfolioOpt documentation, "these
# are not true prices because the initial prices are set to 1, but it behaves as intended when passed to 
# any PyPortfolioOpt method"
returns6 = expected_returns.prices_from_returns(df)
returns6.head(17)

,ITSA4,GOAU4,CMIG3,PETR4,TOTS3,USIM5,LAME4,BBAS3,CIEL3,LIGT3,ABEV3,KLBN11,VVAR3,CVCB3,GOLL4,EMBR3,CSNA3
Date,,,,,,,,,,,,,,,,,
2015-10-26,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
2015-10-27,5.301952e+00,3.516463e+00,5.774228e+00,8.050539e+00,3.404461e+01,3.948654e+00,1.544958e+01,1.405875e+01,2.280339e+01,1.322543e+01,1.839465e+01,2.005384e+01,4.310584e+00,1.393201e+01,4.560000e+00,2.799914e+01,5.082797e+00
2015-10-28,2.795447e+01,1.213264e+01,3.380962e+01,6.742516e+01,1.126252e+03,1.509397e+01,2.383167e+02,1.945444e+02,5.128312e+02,1.734891e+02,3.356173e+02,3.981621e+02,1.639555e+01,2.006314e+02,2.079360e+01,7.831377e+02,2.549739e+01
2015-10-29,1.455775e+02,4.094229e+01,1.943113e+02,5.434345e+02,3.652457e+04,5.623357e+01,3.472793e+03,2.572436e+03,1.105553e+04,2.269016e+03,5.934858e+03,7.773962e+03,5.698243e+01,2.874492e+03,9.336326e+01,2.207144e+04,1.166913e+02
2015-10-30,7.632658e+02,1.416481e+02,1.133808e+03,4.430397e+03,1.214844e+06,2.078655e+02,5.018673e+04,3.496839e+04,2.337626e+05,3.074104e+04,1.054176e+05,1.520468e+05,2.212680e+02,4.144771e+04,4.248029e+02,6.271821e+05,5.369539e+02
2015-11-03,4.141254e+03,5.182009e+02,6.922028e+03,3.928397e+04,4.304494e+07,7.986109e+02,7.745567e+05,5.142237e+05,5.101250e+06,4.200931e+05,1.919681e+06,3.137214e+06,9.251970e+02,6.128785e+05,1.979581e+03,1.851516e+07,2.791610e+03
2015-11-04,2.259124e+04,1.846746e+03,4.333490e+04,3.337502e+05,1.482924e+09,3.052739e+03,1.182941e+07,7.590243e+06,1.096905e+08,5.884603e+06,3.495788e+07,6.419982e+07,3.822574e+03,9.118836e+06,9.304032e+03,5.264858e+08,1.428185e+04
2015-11-05,1.248366e+05,6.581370e+03,2.765625e+05,2.844776e+06,5.293573e+10,1.163967e+04,1.843785e+08,1.171241e+08,2.403986e+09,8.012854e+07,6.322946e+08,1.292051e+09,1.431136e+04,1.365146e+08,4.465935e+04,1.510864e+10,7.294727e+04
2015-11-06,6.765909e+05,2.283183e+04,1.666035e+06,2.348259e+07,1.880145e+12,4.291275e+04,2.756549e+09,1.724201e+09,5.203670e+10,1.055816e+09,1.116447e+10,2.604685e+10,5.358041e+04,1.980970e+09,2.063262e+05,4.268372e+11,3.756189e+05
